<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Software-and-conventions" data-toc-modified-id="Software-and-conventions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Software and conventions</a></span></li></ul></li><li><span><a href="#Recap-of-lessons-1-and-2,-,-compiling-and-running-WRF-Hydro" data-toc-modified-id="Recap-of-lessons-1-and-2,-,-compiling-and-running-WRF-Hydro-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Recap of lessons 1 and 2, , compiling and running WRF-Hydro</a></span><ul class="toc-item"><li><span><a href="#Compiling-WRF-Hydro" data-toc-modified-id="Compiling-WRF-Hydro-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Compiling WRF-Hydro</a></span></li><li><span><a href="#Creating-the-simulation-directory-and-running-a-baseline-WRF-Hydro-simulation" data-toc-modified-id="Creating-the-simulation-directory-and-running-a-baseline-WRF-Hydro-simulation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Creating the simulation directory and running a baseline WRF-Hydro simulation</a></span></li></ul></li><li><span><a href="#National-Water-Model-configuration-(NWM)" data-toc-modified-id="National-Water-Model-configuration-(NWM)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>National Water Model configuration (NWM)</a></span><ul class="toc-item"><li><span><a href="#Create-a-NWM-simulation-directory" data-toc-modified-id="Create-a-NWM-simulation-directory-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create a NWM simulation directory</a></span></li></ul></li></ul></div>

# Lesson 5 - Exploring other configurations
## Overview
In lesson 4 we covered editign namelist options to change WRF-Hydro run-time options. In this lesson, we will experiment with editing some parameter files and explore the *NWM* and *Reach* configuations. 

**NOTE: If you have not completed lessons 4, please stop and do so now.**

### Software and conventions
The easiest way to run these lessons is via the [wrfhydro/training](https://hub.docker.com/r/wrfhydro/training/) Docker container, which has all software dependencies and data pre-installed.

For a complete description of the software environment used for this training please see [Getting started]().

You may either execute commands by running each cell of this notebook. Alternatively, you may open a terminal in Jupyter Lab by selecting `New -> Terminal` in your `Home` tab of Jupyter Lab and input the commands manually if you prefer. 

All paths used in this lesson assume that the lesson materials are located under your home directory in a folder named `wrf-hydro-training`. If your materials are located in another directory, you will not be able to run the commands in this notebook inside Jupyter and will need to type them manually in your terminal session. 

## Recap of lessons 1 and 2, , compiling and running WRF-Hydro
In this section, we will quickly recap the commands issued in Lessons 1 and 2 to compile WRF-Hydro and create our simulation directory. We will use the prepared domain for the remaining lessons, but feel free to replace the prepared domain with the domain you created in the geospatial hands-on training session. **NOTE: Your domain directory structure and filenames must match the prepared domain EXACTLY**

### Compiling WRF-Hydro
Below are the commands to compile WRF-Hydro for the Gridded configuration.

In [ ]:
%%bash
# Change to the trunk/NDHMS directory and configure for gfort
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/; ./configure 2

# Make a copy of the template environment variable file, setEnvars.sh
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/setEnvar.sh \
~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/configureEnvars.sh

**Set compile-time options**

Edit the `configureEnvars.sh` script to set environment variables required by the compile script. Your `configureEnvars.sh` should look like the bash script below when you are finished.  

---

```bash
#!/bin/bash
# WRF-Hydro compile time options

# This is a WRF environment variable. Always set to 1=On for compiling WRF-Hydro.
export WRF_HYDRO=1

# Enhanced diagnostic output for debugging: 0=Off, 1=On.
export HYDRO_D=1

# Spatially distributed parameters for NoahMP: 0=Off, 1=On.
export SPATIAL_SOIL=1  

# RAPID model: 0=Off, 1=On.
export WRF_HYDRO_RAPID=0

# Large netcdf file support: 0=Off, 1=On.
export WRFIO_NCD_LARGE_FILE_SUPPORT=1

# WCOSS file units: 0=Off, 1=On. 
export NCEP_WCOSS=0

# Streamflow nudging: 0=Off, 1=On.
export WRF_HYDRO_NUDGING=0
```

---

**Compile WRF-Hydro in uncoupled mode**

In [ ]:
%%bash
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/; \
./compile_offline_NoahMP.sh configureEnvars.sh >> compile.log 2>&1

tail -13 ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/compile.log

**We have successfully compiled WRF-Hydro and we will use this binary and *.TBL files for the remaining simulations in this lesson**

### Creating the simulation directory and running a baseline WRF-Hydro simulation
Below are the commands from lesson 2 to create the simulation direcory and run a WRF-Hydro simulation. This simulation will be run exactly as was done in Lesson 2 and will serve as our baseline simulation. We will modify run-time options in subsequent simulations in the lesson and compare the outputs.

In [7]:
%%bash
# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson5/run_gridded_baseline

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline

# Create symbolic links to large domain files
cp -as $HOME/wrf-hydro-training/example_case/FORCING \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline
cp -as $HOME/wrf-hydro-training/example_case/Gridded/DOMAIN \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline
cp -as $HOME/wrf-hydro-training/example_case/Gridded/RESTART \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline

# Copy namelist files
cp ~/wrf-hydro-training/example_case/Gridded/namelist.hrldas \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline
cp ~/wrf-hydro-training/example_case/Gridded/hydro.namelist \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline

# Run the simulation
cd ~/wrf-hydro-training/output/lesson5/run_gridded_baseline
mpirun -np 4 ./wrf_hydro.exe >> run.log 2>&1

In [8]:
%%bash
# Check that it finished successfully
tail -1 ~/wrf-hydro-training/output/lesson5/run_gridded_baseline/diag_hydro.00000

 The model finished successfully.......


## National Water Model configuration (NWM)
In this section we will discuss the differences between the National Water Model (NWM) configuration and the baseline Gridded configuration. Afterwards, we will run a NWM simulation and visualize some of the differences. 

### Create a NWM simulation directory

In [10]:
%%bash
# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson5/run_nwm

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL \
~/wrf-hydro-training/output/lesson5/run_nwm
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe \
~/wrf-hydro-training/output/lesson5/run_nwm

# Create symbolic links to large domain files
cp -as $HOME/wrf-hydro-training/example_case/FORCING \
~/wrf-hydro-training/output/lesson5/run_nwm
cp -as $HOME/wrf-hydro-training/example_case/NWM/DOMAIN \
~/wrf-hydro-training/output/lesson5/run_nwm
cp -as $HOME/wrf-hydro-training/example_case/NWM/RESTART \
~/wrf-hydro-training/output/lesson5/run_nwm

# Copy namelist files
cp ~/wrf-hydro-training/example_case/NWM/namelist.hrldas \
~/wrf-hydro-training/output/lesson5/run_nwm
cp ~/wrf-hydro-training/example_case/NWM/hydro.namelist \
~/wrf-hydro-training/output/lesson5/run_nwm

In [12]:
%%bash
# Run the simulation
cd ~/wrf-hydro-training/output/lesson5/run_nwm
mpirun -np 4 ./wrf_hydro.exe >> run.log 2>&1